In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys
import sys
from torchsummary import summary
# sys.path.append('/home/po/TM5/Pointnet_Pointnet2_pytorch')
sys.path.append('/home/po/TM5/s4g-release/inference/grasp_proposal/network_models')
sys.path.append('/home/po/TM5/graspnetAPI/graspnetAPI')
sys.path.append('/home/po/TM5')
from nn_utils.mlp import SharedMLP
from pointnet2_utils.modules import PointNetSAModule, PointnetFPModule, PointNetSAAvgModule
from nn_utils.functional import smooth_cross_entropy
sys.path

Please compile source files before using functions CUDA extension.


['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/pointnet2-0.0.0-py3.6-linux-x86_64.egg',
 '/usr/local/lib/python3.6/dist-packages/knn_pytorch-0.1-py3.6-linux-x86_64.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/po/.ipython',
 '/home/po/TM5/s4g-release/inference/grasp_proposal/network_models',
 '/home/po/TM5/graspnetAPI/graspnetAPI',
 '/home/po/TM5']

In [2]:
sys.path.append('/home/po/TM5/graspnet-baseline')
import scipy.io as scio
from dataset.graspnet_dataset1 import GraspNetDataset, collate_fn, load_grasp_labels
from PIL import Image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2021-11-30 14:45:30,046 - rigid_transformations - Failed to import geometry msgs in rigid_transformations.py.
WARNING - 2021-11-30 14:45:30,047 - rigid_transformations - Failed to import ros dependencies in rigid_transforms.py
WARNING - 2021-11-30 14:45:30,048 - rigid_transformations - autolab_core not installed as catkin package, RigidTransform ros methods will be unavailable


In [3]:
class PointNet2(nn.Module):
    """PointNet++ part segmentation with single-scale grouping

    PointNetSA: PointNet Set Abstraction Layer
    PointNetFP: PointNet Feature Propagation Layer

    Args:
        score_classes (int): the number of grasp score classes
        num_centroids (tuple of int): the numbers of centroids to sample in each set abstraction module
        radius (tuple of float): a tuple of radius to query neighbours in each set abstraction module
        num_neighbours (tuple of int): the numbers of neighbours to query for each centroid
        sa_channels (tuple of tuple of int): the numbers of channels within each set abstraction module
        fp_channels (tuple of tuple of int): the numbers of channels for feature propagation (FP) module
        num_fp_neighbours (tuple of int): the numbers of nearest neighbor used in FP
        seg_channels (tuple of int): the numbers of channels in segmentation mlp
        dropout_prob (float): the probability to dropout input features

    References:
        https://github.com/charlesq34/pointnet2/blob/master/models/pointnet2_part_seg.py

    """
    _SA_MODULE = PointNetSAModule
    _FP_MODULE = PointnetFPModule

    def __init__(self,
                 score_classes,
                 num_centroids=(10240, 1024, 128, 0),
                 radius=(0.2, 0.3, 0.4, -1.0),
                 num_neighbours=(64, 64, 64, -1),
                 sa_channels=((32, 32, 64), (64, 64, 128), (128, 128, 256), (256, 512, 1024)),
                 fp_channels=((256, 256), (256, 128), (128, 128), (64, 64, 64)),
                 num_fp_neighbours=(0, 3, 3, 3),
                 seg_channels=(128,),
                 num_removal_directions=5,
                 dropout_prob=0.5):
        super(PointNet2, self).__init__()

        # Sanity check
        num_sa_layers = len(num_centroids)
        num_fp_layers = len(fp_channels)
        assert len(radius) == num_sa_layers
        assert len(num_neighbours) == num_sa_layers
        assert len(sa_channels) == num_sa_layers
        assert num_sa_layers == num_fp_layers
        assert len(num_fp_neighbours) == num_fp_layers

        # Set Abstraction Layers
        feature_channels = 0
        self.sa_modules = nn.ModuleList()
        for ind in range(num_sa_layers):
            sa_module = self._SA_MODULE(in_channels=feature_channels,
                                        mlp_channels=sa_channels[ind],
                                        num_centroids=num_centroids[ind],
                                        radius=radius[ind],
                                        num_neighbours=num_neighbours[ind],
                                        use_xyz=True)
            self.sa_modules.append(sa_module)
            feature_channels = sa_channels[ind][-1]

        inter_channels = [0]
        inter_channels.extend([x[-1] for x in sa_channels])

        # Feature Propagation Layers
        self.fp_modules = nn.ModuleList()
        feature_channels = inter_channels[-1]
        for ind in range(num_fp_layers):
            fp_module = self._FP_MODULE(in_channels=feature_channels + inter_channels[-2 - ind],
                                        mlp_channels=fp_channels[ind],
                                        num_neighbors=num_fp_neighbours[ind])
            self.fp_modules.append(fp_module)
            feature_channels = fp_channels[ind][-1]

        # MLP
#         self.mlp_seg = SharedMLP(feature_channels, seg_channels, ndim=1, dropout_prob=dropout_prob)
#         self.seg_logit = nn.Conv1d(seg_channels[-1], score_classes, 1, bias=True)

        self.mlp_grasp_eval = SharedMLP(feature_channels + 28, seg_channels, ndim=2, dropout_prob=dropout_prob)
        self.grasp_eval_logit = nn.Conv2d(seg_channels[-1], 1, 1, bias=True)
    
        self.mlp_R = SharedMLP(feature_channels, seg_channels, ndim=1)
        self.R_logit = nn.Conv1d(seg_channels[-1], 4, 1, bias=True)

        self.mlp_t = SharedMLP(feature_channels, seg_channels, ndim=1)
        self.t_logit = nn.Conv1d(seg_channels[-1], 3, 1, bias=True)

#         self.mlp_movable = SharedMLP(feature_channels, seg_channels, ndim=1, dropout_prob=dropout_prob)
#         self.movable_logit = nn.Sequential(
#             nn.Conv1d(seg_channels[-1], num_removal_directions, 1, bias=True),
#             nn.Sigmoid())

        self.init_weights()
    def forward(self, data_batch):
        points = data_batch["point_clouds"]

        xyz = points
        feature = None

        # save intermediate results
        inter_xyz = [xyz]
        inter_feature = [feature]

        # Set Abstraction Layers
        for sa_module in self.sa_modules:
            xyz, feature = sa_module(xyz, feature)
            inter_xyz.append(xyz)
            inter_feature.append(feature)

        # Feature Propagation Layers
        sparse_xyz = xyz
        sparse_feature = feature
        for fp_ind, fp_module in enumerate(self.fp_modules):
            dense_xyz = inter_xyz[-2 - fp_ind]
            dense_feature = inter_feature[-2 - fp_ind]
            fp_feature = fp_module(dense_xyz, sparse_xyz, dense_feature, sparse_feature)
            sparse_xyz = dense_xyz
            sparse_feature = fp_feature

        # MLP
#         x = self.mlp_seg(sparse_feature)
#         logits = self.seg_logit(x)
        
              
        
        
        R = self.mlp_R(sparse_feature)
        R = self.R_logit(R)
        R = F.normalize(R, dim=1)
        # R = toRotMatrix(R)
        # R = euler2RotMatrix(R)

        t = self.mlp_t(sparse_feature)
        t = self.t_logit(t)
        
        local_search_frame = torch.cat([R, t], dim=1).unsqueeze(-1)
        local_search_frame = local_search_frame.repeat(1, 4, 1, 1)
        sparse_feature = sparse_feature.unsqueeze(-1)
        valid_feature = torch.cat([sparse_feature, local_search_frame], dim=1)
        local_search_logit = self.grasp_eval_logit(self.mlp_grasp_eval(valid_feature))
        # t = points + t
        
#         mov = self.mlp_movable(sparse_feature)
#         mov = self.movable_logit(mov)  # (B, 5, N)
        
            
        preds = {
#                 "score": logits,
                "score_pred": local_search_logit,
                 "q_pred": R,
                 "xyz_pred": t,
#                  "movable_logits": mov,
                 }

        return preds

    def init_weights(self):
        # nn_utils.init.zeros_(self.t_logit.weight)
        # nn_utils.init.zeros_(self.t_logit.bias)
        pass

In [ ]:
#(1,4,5->20000)
A = torch.tensor([[[1., 2., 3., 4., 5.],
         [1., 2., 3., 4., 5.],
         [2., 2., 3., 4., 5.],
         [2., 2., 3., 4., 5.]]])
B = torch.tensor([[[2., 2., 3., 4., 5.],
         [5., 2., 3., 4., 5.],
         [4., 2., 3., 4., 5.],
         [3., 2., 3., 4., 5.]]])
C = B-A
C
F.normalize(C**2,dim=1)
# (C**2).mean(1,True)

In [ ]:
loss = nn.L1Loss()
input = torch.tensor([[[1., 2., 3., 4., 5.],
         [1., 2., 3., 4., 5.],
         [2., 2., 3., 4., 5.],
         [2., 2., 3., 4., 5.]]])
target = torch.tensor([[[3.,4., 3., 4., 5.],
         [3., 4., 3., 4., 5.],
         [3., 3., 3., 4., 5.],
         [3., 3., 3., 4., 5.]]])
# output.backward()
print(input)
print(target)
output = loss(input, target)
output

R = torch.ones(1,4,20000)
t = torch.ones(1,3,20000)
sparse_feature = torch.ones(1,256,20000)
local_search_frame = torch.cat([R, t], dim=1).unsqueeze(-1)
local_search_frame.shape
local_search_frame = local_search_frame.repeat(1, 4, 1, 1)
sparse_feature = sparse_feature.unsqueeze(-1)
valid_feature = torch.cat([sparse_feature, local_search_frame], dim=1)
valid_feature.shape
# # model.mlp_grasp_eval(valid_feature.cuda())
local_search_logit = model.grasp_eval_logit(model.mlp_grasp_eval(valid_feature))
local_search_logit.shape

R = torch.tensor([[[1, 0.5000, 0.5000, 0.5000, 0.5000],
         [1, 0.5000, 0.5000, 0.5000, 0.5000],
         [2, 0.5000, 0.5000, 0.5000, 0.5000],
         [2, 0.5000, 0.5000, 0.5000, 0.5000]]])
print(R.shape)
R = torch.nn.functional.normalize(R, dim=1)
R

In [4]:
class PointNet2Loss(nn.Module):
    def __init__(self, label_smoothing=0, neg_weight=0.1):
        super(PointNet2Loss, self).__init__()
        self.label_smoothing = label_smoothing
        self.neg_weight = neg_weight
        self.loss1 = nn.L1Loss()
        self.loss2 = nn.MSELoss()
    def forward(self, preds, labels):
        

        q_label = labels["q_label"]#.permute(0,2,1)#(1,4,20)
        q_pred = preds["q_pred"]
        q_loss = self.loss1(q_pred,q_label)
#         q_loss = ((q_pred - q_label) ** 2).mean(1, True)
        

        # weight loss according to gt_score
        score_label = labels["score_label"]#.permute(0,2,1).unsqueeze(-1)
        score_pred = preds["score_pred"]
        score_loss = self.loss2(score_pred,score_label)
#         score_loss = ((score_pred - score_label) ** 2).mean(1, True)
        

        xyz_label = labels["xyz_label"]#.permute(0,2,1)
        xyz_pred = preds["xyz_pred"]
        xyz_loss = self.loss2(xyz_pred,xyz_label)
#         xyz_loss = ((xyz_pred - xyz_label) ** 2).mean(1, True)
        
    

        loss_dict = {
                    "score_loss": score_loss,
                     "q_loss": q_loss,
                     "xyz_loss": xyz_loss,
                     }

        return loss_dict


In [5]:
class PointNet2Metric(nn.Module):
    def forward(self, preds, labels):
        scene_score_logits = preds["scene_score_logits"]  # (B, C, N2)
        score_classes = scene_score_logits.shape[1]

        scene_score_labels = labels["scene_score_labels"]  # (B, N)

        selected_preds = scene_score_logits.argmax(1).view(-1)
        scene_score_labels = scene_score_labels.view(-1)

        cls_acc = selected_preds.eq(scene_score_labels).float()

        movable_logits = preds["movable_logits"]
        movable_labels = labels["scene_movable_labels"]
        movable_preds = (movable_logits > 0.5).view(-1).int()
        movable_labels = movable_labels.view(-1).int()
        mov_acc = movable_preds.eq(movable_labels).float()

        gt_frame_R = labels["best_frame_R"]
        batch_size, _, num_frame_points = gt_frame_R.shape
        pred_frame_R = preds["frame_R"][:, :, :num_frame_points]
        gt_frame_R = gt_frame_R.transpose(1, 2).contiguous().view(batch_size * num_frame_points, 3, 3)
        gt_frame_R_inv = gt_frame_R.clone()
        gt_frame_R_inv[:, :, 1:] = -gt_frame_R_inv[:, :, 1:]
        pred_frame_R = pred_frame_R.transpose(1, 2).contiguous().view(batch_size * num_frame_points, 3, 3)
        M = torch.bmm(gt_frame_R, pred_frame_R.transpose(1, 2))
        angle = torch.acos(torch.clamp((M[:, 0, 0] + M[:, 1, 1] + M[:, 2, 2] - 1.0) / 2.0, -1.0, 1.0))
        M_inv = torch.bmm(gt_frame_R_inv, pred_frame_R.transpose(1, 2))
        angle_inv = torch.acos(torch.clamp((M_inv[:, 0, 0] + M_inv[:, 1, 1] + M_inv[:, 2, 2] - 1.0) / 2.0, -1.0, 1.0))

        angle_min = torch.stack([angle, angle_inv], dim=1).min(1)[0]
        gt_scene_score = labels["scene_score"][:, :num_frame_points].contiguous().view(-1)
        angle_min = (gt_scene_score * angle_min).mean()

        gt_frame_t = labels["best_frame_t"].view(-1)
        pred_frame_t = preds["frame_t"][:, :, :num_frame_points]
        pred_frame_t = torch.argmax(pred_frame_t, dim=1).view(-1)
        t_acc = pred_frame_t.eq(gt_frame_t).float()

        # t_err = torch.mean(torch.sqrt(((gt_frame_t - pred_frame_t) ** 2).sum(1)))

        return {"cls_acc": cls_acc,
                "mov_acc": mov_acc,
                "R_err": angle_min,
                "t_acc": t_acc,
                }


In [6]:
# def build_pointnet2_cls(cfg):
def build_model(cfg):
    net = PointNet2(
        score_classes=cfg.DATA.SCORE_CLASSES,
        num_centroids=cfg.MODEL.PN2.NUM_CENTROIDS,
        radius=cfg.MODEL.PN2.RADIUS,
        num_neighbours=cfg.MODEL.PN2.NUM_NEIGHBOURS,
        sa_channels=cfg.MODEL.PN2.SA_CHANNELS,
        fp_channels=cfg.MODEL.PN2.FP_CHANNELS,
        num_fp_neighbours=cfg.MODEL.PN2.NUM_FP_NEIGHBOURS,
        seg_channels=cfg.MODEL.PN2.SEG_CHANNELS,
        num_removal_directions=cfg.DATA.NUM_REMOVAL_DIRECTIONS,
        dropout_prob=cfg.MODEL.PN2.DROPOUT_PROB,
    )

    loss_func = PointNet2Loss(
        label_smoothing=cfg.MODEL.PN2.LABEL_SMOOTHING,
        neg_weight=cfg.MODEL.PN2.NEG_WEIGHT,
    )
    metric = PointNet2Metric()

    return net, loss_func, metric


In [7]:
root = '/home/po/TM5/graspnetAPI'
valid_obj_idxs, grasp_labels = load_grasp_labels(root)
train_dataset = GraspNetDataset(root, valid_obj_idxs, grasp_labels, split='train', remove_outlier=True, remove_invisible=True, num_points=20000)
print(len(train_dataset))

Loading data path and collision labels...: 100%|██████████| 15/15 [00:02<00:00,  5.64it/s]

3840


In [ ]:
# grasp_list = train_dataset.__getitem__(0)['grasp_list']
# point_clouds = train_dataset.__getitem__(0)['point_clouds']

In [ ]:
# print(grasp_list.shape,point_clouds.shape)

In [8]:
# train.py
import os
import sys
import numpy as np
from datetime import datetime
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
sys.path.append('/home/po/TM5/graspnet-baseline/pointnet2')

from pytorch_utils import BNMomentumScheduler

# Init datasets and dataloaders 
def my_worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    pass

In [9]:
TRAIN_DATALOADER = DataLoader(train_dataset, batch_size=1, shuffle=True,
    num_workers=4, worker_init_fn=my_worker_init_fn, collate_fn=collate_fn)

In [26]:
tmp_dic = next(iter(TRAIN_DATALOADER))

In [27]:
tmp_dic.keys()

dict_keys(['point_clouds', 'cloud_colors', 'xyz_label', 'q_label', 'score_label'])

In [ ]:
tmp_dic["point_clouds"].permute(0,2,1).float().cpu()

In [10]:
# proposal_test.py
import sys,os
sys.path.append('/home/po/TM5/s4g-release/inference')
import numpy as np
import open3d
import time
import torch
import torch.nn as nn
from grasp_proposal.cloud_processor.cloud_processor import CloudPreProcessor
from grasp_proposal.configs.yacs_config import load_cfg_from_file
# from grasp_proposal.network_models.models.build_model import build_model
from grasp_proposal.utils.checkpoint import CheckPointer
from grasp_proposal.utils.file_logger_cls import loggin_to_file
from grasp_proposal.utils.grasp_visualizer import GraspVisualizer
from grasp_proposal.utils.logger import setup_logger, MetricLogger

In [11]:
# proposal_test.py
# load_static batch data
# 
cfg_path = "/home/po/TM5/s4g-release/inference/grasp_proposal/configs/curvature_model.yaml"
cfg = load_cfg_from_file(cfg_path)
cfg.defrost()
# cfg.TEST.WEIGHT = cfg.TEST.WEIGHT.replace("${PROJECT_HOME}", os.path.join(os.getcwd(), "../"))
# cfg.TEST.WEIGHT = '/home/po/TM5/s4g-release/inference/trained_models/curvature_model.pth'
cfg.TEST.WEIGHT = cfg.TEST.WEIGHT.replace("${PROJECT_HOME}", os.path.join('/home/po/TM5/s4g-release/inference'))
cfg.freeze()
assert cfg.TEST.BATCH_SIZE == 1

output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

logger = setup_logger("S4G", output_dir, "unit_test")
logger.info("Using {} of GPUs".format(torch.cuda.device_count()))
logger.info("Load config file from {}".format(cfg_path))
logger.debug("Running with config \n {}".format(cfg))

model, loss_func, _ = build_model(cfg)
loss_func


2021-11-30 14:46:09,214 S4G INFO: Using 2 of GPUs
2021-11-30 14:46:09,215 S4G INFO: Load config file from /home/po/TM5/s4g-release/inference/grasp_proposal/configs/curvature_model.yaml


PointNet2Loss(
  (loss1): L1Loss()
  (loss2): MSELoss()
)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
loss = {'a':1,'b':1,'c':1}

sum(loss.values())

In [12]:
lr_decay_steps = '8,12,16'
lr_decay_rates = '0.1,0.1,0.1'
EPOCH_CNT = 0
LR_DECAY_STEPS = [int(x) for x in lr_decay_steps.split(',')]
LR_DECAY_RATES = [float(x) for x in lr_decay_rates.split(',')]
assert(len(LR_DECAY_STEPS)==len(LR_DECAY_RATES))

In [13]:
def train_one_epoch():
    adjust_learning_rate(optimizer, EPOCH_CNT)
    bnm_scheduler.step() # decay BN momentum
    # set model to training mode
    model.train()
    for batch_idx, batch_data_label in enumerate(tqdm(TRAIN_DATALOADER)):
        data_batch = {
        "point_clouds": batch_data_label["point_clouds"].permute(0,2,1).float().cuda(),
        "xyz_label": batch_data_label["xyz_label"].permute(0,2,1).float().cuda(),
        "q_label": batch_data_label["q_label"].permute(0,2,1).float().cuda(),
        "score_label": batch_data_label["score_label"].permute(0,2,1).unsqueeze(-1).float().cuda(),
        }
        predictions = model(data_batch)
    #     print(batch_idx)
    #     print('pred',predictions['xyz_pred'].shape,predictions['q_pred'].shape,predictions['score_pred'].shape)
    #     print('dlabel',data_batch['point_clouds'].shape,data_batch['xyz_label'].shape,data_batch['q_label'].shape,data_batch['score_label'].shape)
        loss = loss_func(predictions, data_batch)
        sum(loss.values()).backward()   
        optimizer.step()
        optimizer.zero_grad()
    return loss
def evaluate_one_epoch():
    model.train()
    for batch_idx, batch_data_label in enumerate(TRAIN_DATALOADER):
        data_batch = {
        "point_clouds": batch_data_label["point_clouds"].permute(0,2,1).float().cuda(),
        "xyz_label": batch_data_label["xyz_label"].permute(0,2,1).float().cuda(),
        "q_label": batch_data_label["q_label"].permute(0,2,1).float().cuda(),
        "score_label": batch_data_label["score_label"].permute(0,2,1).unsqueeze(-1).float().cuda(),
        }
        with torch.no_grad():
            predictions = model(data_batch)
    #     print(batch_idx)
    #     print('pred',predictions['xyz_pred'].shape,predictions['q_pred'].shape,predictions['score_pred'].shape)
    #     print('dlabel',data_batch['point_clouds'].shape,data_batch['xyz_label'].shape,data_batch['q_label'].shape,data_batch['score_label'].shape)
        loss = loss_func(predictions, data_batch)
        sum(loss.values())
        
    return mean_loss

In [24]:
from tqdm import tqdm
learning_rate = 0.001
def get_current_lr(epoch):
    lr = learning_rate
    for i,lr_decay_epoch in enumerate(LR_DECAY_STEPS):
        if epoch >= lr_decay_epoch:
            lr *= LR_DECAY_RATES[i]
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
LOG_FOUT = open(os.path.join('/home/po/TM5/s4g-release/inference/grasp_proposal/network_models/models', 'log_train.txt'), 'a')

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
if torch.cuda.device_count() > 10:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()
bn_decay_step = 2
bn_decay_rate = 0.5
start_epoch = 0
BN_MOMENTUM_INIT = 0.5
BN_MOMENTUM_MAX = 0.001
bn_lbmd = lambda it: max(BN_MOMENTUM_INIT * bn_decay_rate**(int(it / bn_decay_step)), BN_MOMENTUM_MAX)
bnm_scheduler = BNMomentumScheduler(model, bn_lambda=bn_lbmd, last_epoch=start_epoch-1)

min_loss = 1e10
loss = 0
global EPOCH_CNT
trained_model_path = output_dir
check_pointer = CheckPointer(model, save_dir=trained_model_path, logger=logger)
for epoch in range(10):
    EPOCH_CNT = epoch
    log_string('**** EPOCH %03d ****' % (epoch))
    log_string('Current learning rate: %f'%(get_current_lr(epoch)))
    log_string('Current BN decay momentum: %f'%(bnm_scheduler.lmbd(bnm_scheduler.last_epoch)))
    log_string(str(datetime.now()))
    # Reset numpy seed.
    # REF: https://github.com/pytorch/pytorch/issues/5059
    np.random.seed()
    loss = train_one_epoch()
    print(epoch,loss)
#     loss = evaluate_one_epoch()
    

**** EPOCH 000 ****
Current learning rate: 0.001000
Current BN decay momentum: 0.500000
2021-11-30 14:53:12.722039


100%|██████████| 3840/3840 [34:31<00:00,  1.85it/s]


0 {'score_loss': tensor(0.0162, device='cuda:0', grad_fn=<MseLossBackward0>), 'q_loss': tensor(0.2579, device='cuda:0', grad_fn=<L1LossBackward0>), 'xyz_loss': tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward0>)}
**** EPOCH 001 ****
Current learning rate: 0.001000
Current BN decay momentum: 0.500000
2021-11-30 15:27:44.309269


100%|██████████| 3840/3840 [34:28<00:00,  1.86it/s]


1 {'score_loss': tensor(0.0130, device='cuda:0', grad_fn=<MseLossBackward0>), 'q_loss': tensor(0.2556, device='cuda:0', grad_fn=<L1LossBackward0>), 'xyz_loss': tensor(0.0011, device='cuda:0', grad_fn=<MseLossBackward0>)}
**** EPOCH 002 ****
Current learning rate: 0.001000
Current BN decay momentum: 0.500000
2021-11-30 16:02:12.646190


...

7 {'score_loss': tensor(0.0157, device='cuda:0', grad_fn=<MseLossBackward0>), 'q_loss': tensor(0.2557, device='cuda:0', grad_fn=<L1LossBackward0>), 'xyz_loss': tensor(0.0014, device='cuda:0', grad_fn=<MseLossBackward0>)}
**** EPOCH 008 ****
Current learning rate: 0.000100
Current BN decay momentum: 0.062500
2021-11-30 19:28:18.121216


100%|██████████| 3840/3840 [34:21<00:00,  1.86it/s]


8 {'score_loss': tensor(0.0140, device='cuda:0', grad_fn=<MseLossBackward0>), 'q_loss': tensor(0.2553, device='cuda:0', grad_fn=<L1LossBackward0>), 'xyz_loss': tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward0>)}
**** EPOCH 009 ****
Current learning rate: 0.000100
Current BN decay momentum: 0.031250
2021-11-30 20:02:39.187106


100%|██████████| 3840/3840 [34:20<00:00,  1.86it/s]

9 {'score_loss': tensor(0.0143, device='cuda:0', grad_fn=<MseLossBackward0>), 'q_loss': tensor(0.2554, device='cuda:0', grad_fn=<L1LossBackward0>), 'xyz_loss': tensor(0.0014, device='cuda:0', grad_fn=<MseLossBackward0>)}


In [25]:
torch.save(model.state_dict(), 'graspv1.pth')


In [30]:
#load_state_dict(torch.load('graspv1.pth'))
model, _, _ = build_model(cfg)
if torch.cuda.device_count() > 13:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()
if torch.cuda.device_count() > 10:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()
model.load_state_dict(torch.load('graspv1.pth'))

model.eval()
with torch.no_grad():
    data_batch = {
        "point_clouds": tmp_dic["point_clouds"].permute(0,2,1).float().cuda(),
        "xyz_label": tmp_dic["xyz_label"].permute(0,2,1).float().cuda(),
        "q_label": tmp_dic["q_label"].permute(0,2,1).float().cuda(),
        "score_label": tmp_dic["score_label"].permute(0,2,1).unsqueeze(-1).float().cuda(),
        }
#     tac = time.time()
#     data_time = tac - tic
    predictions = model(data_batch)
#     tic = time.time()
#     batch_time = tic - tac
#     with open("inference_time_{}.txt".format("ours"), "a+") as f:
#         f.write("{:.4f}\n".format(batch_time * 1000.0))
#     meters.update(time=batch_time, data=data_time)

#     logger.info(meters.delimiter.join(["{meters}", ]).format(meters=str(meters), ))

#     top_poses, score = loggin_to_file(data_batch, predictions, 0, output_dir, prefix="test", with_label=False)
#     visualizer = GraspVisualizer(pcd)
#     visualizer.add_multiple_poses(top_poses)
#     visualizer.visualize()

In [ ]:
def draw_grasp_scene(data_batch,predictions):
    

In [37]:

print(tmp_dic['point_clouds'].squeeze(0).shape,tmp_dic['cloud_colors'].squeeze(0).shape)

torch.Size([20000, 3]) torch.Size([20000, 3])


In [34]:
#test open3d 
import open3d as o3d
import numpy as np
import cv2
import copy

In [ ]:
def plot_gripper_pro_max(center, R, width, depth, score=1, color=None):
    '''
    Author: chenxi-wang
    
    **Input:**

    - center: numpy array of (3,), target point as gripper center

    - R: numpy array of (3,3), rotation matrix of gripper

    - width: float, gripper width

    - score: float, grasp quality score

    **Output:**

    - open3d.geometry.TriangleMesh
    '''
    x, y, z = center
    height=0.004
    finger_width = 0.004
    tail_length = 0.04
    depth_base = 0.02
    
    if color is not None:
        color_r, color_g, color_b = color
    else:
        color_r = score # red for high score
        color_g = 0
        color_b = 1 - score # blue for low score
    
    left = create_mesh_box(depth+depth_base+finger_width, finger_width, height)
    right = create_mesh_box(depth+depth_base+finger_width, finger_width, height)
    bottom = create_mesh_box(finger_width, width, height)
    tail = create_mesh_box(tail_length, finger_width, height)

    left_points = np.array(left.vertices)
    left_triangles = np.array(left.triangles)
    left_points[:,0] -= depth_base + finger_width
    left_points[:,1] -= width/2 + finger_width
    left_points[:,2] -= height/2

    right_points = np.array(right.vertices)
    right_triangles = np.array(right.triangles) + 8
    right_points[:,0] -= depth_base + finger_width
    right_points[:,1] += width/2
    right_points[:,2] -= height/2

    bottom_points = np.array(bottom.vertices)
    bottom_triangles = np.array(bottom.triangles) + 16
    bottom_points[:,0] -= finger_width + depth_base
    bottom_points[:,1] -= width/2
    bottom_points[:,2] -= height/2

    tail_points = np.array(tail.vertices)
    tail_triangles = np.array(tail.triangles) + 24
    tail_points[:,0] -= tail_length + finger_width + depth_base
    tail_points[:,1] -= finger_width / 2
    tail_points[:,2] -= height/2

    vertices = np.concatenate([left_points, right_points, bottom_points, tail_points], axis=0)
    vertices = np.dot(R, vertices.T).T + center
    triangles = np.concatenate([left_triangles, right_triangles, bottom_triangles, tail_triangles], axis=0)
    colors = np.array([ [color_r,color_g,color_b] for _ in range(len(vertices))])

    gripper = o3d.geometry.TriangleMesh()
    gripper.vertices = o3d.utility.Vector3dVector(vertices)
    gripper.triangles = o3d.utility.Vector3iVector(triangles)
    gripper.vertex_colors = o3d.utility.Vector3dVector(colors)
    return gripper

In [40]:
from graspnetAPI import GraspNet, Grasp, GraspGroup

g = Grasp()

In [42]:
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
# axis_pcd_tx = copy.deepcopy(axis_pcd).translate((1.3,0,0))
# points = np.array([[0.1, 0.1, 0.1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
# colors = [[1, 1, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
points = tmp_dic['point_clouds'].squeeze(0)
colors = tmp_dic['cloud_colors'].squeeze(0)
test_pcd = open3d.geometry.PointCloud()  # 定义点云
geometries = [axis_pcd,test_pcd,g.to_open3d_geometry()]
# geometries.append(axis_pcd)
test_pcd.points = o3d.utility.Vector3dVector(points)  # 定义点云坐标位置
test_pcd.colors = o3d.utility.Vector3dVector(colors)  # 定义点云的颜色

open3d.visualization.draw_geometries(geometries)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
if torch.cuda.device_count() > 10:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()
model.train()

for batch_idx, batch_data_label in enumerate(TRAIN_DATALOADER):
    data_batch = {
    "point_clouds": batch_data_label["point_clouds"].permute(0,2,1).float().cuda(),
    "xyz_label": batch_data_label["xyz_label"].permute(0,2,1).float().cuda(),
    "q_label": batch_data_label["q_label"].permute(0,2,1).float().cuda(),
    "score_label": batch_data_label["score_label"].permute(0,2,1).unsqueeze(-1).float().cuda(),
    }
    predictions = model(data_batch)
    print(batch_idx)
#     print('pred',predictions['xyz_pred'].shape,predictions['q_pred'].shape,predictions['score_pred'].shape)
#     print('dlabel',data_batch['point_clouds'].shape,data_batch['xyz_label'].shape,data_batch['q_label'].shape,data_batch['score_label'].shape)
    loss = loss_func(predictions, data_batch)
    sum(loss.values()).backward()   
    optimizer.step()
    optimizer.zero_grad()
#     model.save
#     print(loss['xyz_loss'],loss['q_loss'],loss['score_loss'])


In [ ]:
# loss = loss(predictions, data_batch1)
loss_func()

In [ ]:
if torch.cuda.device_count() > 10:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()
    
model.eval()
data_batch = {
"point_clouds": tmp_dic['point_clouds'].permute(0,2,1).float().cuda(),
"xyz_label": tmp_dic['xyz_label'].permute(0,2,1).float().cuda(),
"q_label": tmp_dic['q_label'].permute(0,2,1).float().cuda(),
"score_label": tmp_dic['score_label'].permute(0,2,1).unsqueeze(-1).float().cuda(),
}
predictions = model(data_batch)
# print(batch_idx)
#     print('pred',predictions['xyz_pred'].shape,predictions['q_pred'].shape,predictions['score_pred'].shape)
#     print('dlabel',data_batch['point_clouds'].shape,data_batch['xyz_label'].shape,data_batch['q_label'].shape,data_batch['score_label'].shape)
loss = loss(predictions, data_batch)
# sum(loss.values()).backward()   

In [ ]:
loss

In [ ]:
print(tmp_dic['point_clouds'].shape,tmp_dic['xyz_label'].shape,tmp_dic['q_label'].shape,tmp_dic['score_label'].shape)

In [ ]:
model

In [ ]:
print(predictions.keys())
print(predictions['score_pred'].shape,predictions['q_pred'].shape,predictions['xyz_pred'].shape)
print(data_batch['score_label'].shape,data_batch['q_label'].permute(0,2,1).shape,data_batch['xyz_label'].permute(0,2,1).shape)

In [ ]:
# A = torch.tensor([[[2., 2., 2.],
#          [3., 3., 3.]],

#         [[4., 4., 4.],
#          [5., 5., 6.]]])
# B = torch.tensor([[[1., 1., 1.],
#          [1., 1., 1.]],

#         [[1., 1., 1.],
#          [1., 1., 1.]]])
A = torch.ones(1,4,20000)
B = torch.ones(1,4,20000)
D = (A -B)**2
E = ((A -B)**2).mean(1,True)
print(D.shape,E.shape)

In [ ]:
q_label = data_batch["q_label"].permute(0,2,1)#(1,4,20)
q_pred = predictions["q_pred"]
q_loss = ((q_pred - q_label) ** 2).mean(1, True)

q_loss.shape
# # weight loss according to gt_score
# score_label = labels["score_label"].permute(0,2,1).unsqueeze(-1)
# score_pred = preds["score_pred"]
# score_loss = ((score_pred - score_label) ** 2).mean(1, True)


# xyz_label = labels["xyz_label"].permute(0,2,1)
# xyz_pred = preds["xyz_pred"]
# xyz_loss = ((xyz_pred - xyz_label) ** 2).mean(1, True)


q_label = q_label.permute(0,2,1)
q_label.shape
xyz_label.permute(0,2,1).shape
score_label.shape

In [ ]:
print(predictions['score_pred'].shape,predictions['q_pred'].shape,predictions['xyz_pred'].shape)

In [ ]:
pn2_loss_fn = loss().cuda()
loss = pn2_loss_fn(predictions, data_batch)
loss['xyz_loss'].shape

In [ ]:
sys.path

In [ ]:
sys.path.append('/home/po/TM5/s4g-release/inference')
import numpy as np
import open3d
import time
import torch
import torch.nn as nn
from grasp_proposal.cloud_processor.cloud_processor import CloudPreProcessor
from grasp_proposal.configs.yacs_config import load_cfg_from_file
from grasp_proposal.network_models.models.build_model import build_model
from grasp_proposal.utils.checkpoint import CheckPointer
from grasp_proposal.utils.file_logger_cls import loggin_to_file
from grasp_proposal.utils.grasp_visualizer import GraspVisualizer
from grasp_proposal.utils.logger import setup_logger, MetricLogger

In [ ]:
# single_training_data = np.load("/home/po/TM5/s4g-release/2638_view_0.p", allow_pickle=True)
# cloud_array = single_training_data["point_cloud"]
# cloud = CloudPreProcessor(open3d.geometry.PointCloud(open3d.utility.Vector3dVector(cloud_array.T)), False)
# # do not filter workspace here since training data
# cloud.voxelize()
# cloud.remove_outliers()
# points = np.asarray(cloud.pcd.points)
# if points.shape[0] > 25600:
#     random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=False)
# else:
#     random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=True)

# points = points[random_index, :]
# C = torch.tensor(points, dtype=torch.float32).unsqueeze(0).transpose(1, 2)
# B = torch.tensor(points, dtype=torch.float32).unsqueeze(0)
# A = torch.tensor(points, dtype=torch.float32)
# # data_batch = {"scene_points": torch.tensor(points, dtype=torch.float32).unsqueeze(0).transpose(1, 2)}
# # print(data_batch,cloud.pcd)
# print(A.shape,B.shape,C.shape)

In [ ]:
# single_training_data['point_cloud'].shape

In [ ]:
R = torch.ones(1, 9, 25600) 
t = torch.ones(1, 3, 25600)
local_search_frame = torch.cat([R, t], dim=1).unsqueeze(-1)
local_search_frame = local_search_frame.repeat(1, 4, 1, 1)
# sparse_feature = sparse_feature.unsqueeze(-1)
# valid_feature = torch.cat([sparse_feature, local_search_frame], dim=1)
# local_search_logit = self.grasp_eval_logit(self.mlp_grasp_eval(valid_feature))

In [ ]:
local_search_frame.shape

In [ ]:
local_search_frame1.shape

In [ ]:

print(torch.__version__)